In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ROOT='''/content/drive/MyDrive/Colab Notebooks/googledrive/attentionprediction'''

Mounted at /content/drive


In [ ]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [ ]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_ivsn','h_gbvs','h_deepgaze','h_chatgpt']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

conn = sqlite3.connect(ROOT+"/FV_ChatGPT/ec2_db/turing_chatgpt_fv.db")
cursor = conn.cursor()
cursor.execute("select * from newdb;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'ivsn':1,'gbvs':1,'deepgaze':1,'swap':1,'catch':1,'chatgpt':1}
modelname_dict= {'ivsn':'IVSN','gbvs':'GBVS','deepgaze':'DeepGazeIII','chatgpt':"ChatGPT",'swap':'SwappedBackground','catch':'CatchTrial'}
count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []
dummy_marks_overall = []

confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue

        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 40
        threshold = nTrials*(7/12) # change to number of total trials
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST' and \
                dat['trialdata']['object response'] == dat['trialdata']['correct object']:
                dummy_mark += 1
        if dummy_mark < threshold:
            continue
        dummy_marks_overall.append(dummy_mark)

        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))



        participant_count += 1

        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                count += 1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trial_data = trial_data['trialData']
#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    model = 'catch'
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    model = 'swap'
                else:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
#                         print(trial_data['imageID'])
                        model = trial_data['trialData']['imageID'].split('/')[-2].split('_')[-1] if 'chatgpt' not in trial_data['trialData']['imageID'] else 'chatgpt'
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
#                     marks.append(1 if trial_data['correct response'] == trial_data['choose response'] else 0)
#                     marks_conf.append(deepcopy(confmatrices['h_machine']))
#                 else:

#                 else:
#                     print('catch trial')



                # clear inconsistency
                trial_data['trialData']['response_gender'] = trial_data['trialData']['gender response']
                del trial_data['trialData']['gender response']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

                trial_data['trialData']['response_object'] = trial_data['trialData']['object response'].lower()
                del trial_data['trialData']['object response']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['correct object'].lower()
                del trial_data['trialData']['correct object']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
                trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_FV_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)

        print(confmatrices)
        print('catch',catch_trial_mat)
        print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-03-27 06:59:36
[Time Used] 8.2 mins
[Questionnaire] {'country': 'United States', 'age': '46-50', 'engagement': '9', 'difficulty': '8', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '3', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 29/40
{'h_machine': array([[ 9., 11.],
       [10., 10.]]), 'h_ivsn': array([[2., 3.]]), 'h_gbvs': array([[4., 1.]]), 'h_deepgaze': array([[3., 2.]]), 'h_chatgpt': array([[1., 4.]])}
catch [[0. 0.]]
swap [[0. 0.]]
[Finished Date] 2025-03-27 07:01:36
[Time Used] 7.0 mins
[Questionnaire] {'country': 'United States', 'age': '46-50', 'engagement': '8', 'difficulty': '5', 'mode': 'live', 'gender': 'male', 'education': 'master', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 27/40
{'h_machine': array([[ 9., 11.],
       [ 9., 11.]]), 'h_ivsn': array([[4., 1.]]), 'h_gbvs': array([[2., 3.]]), 'h_deepgaze': array([[0., 5.]]), 'h_chatgpt': array([[3., 

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/attention_prediction_task/Plot/Data_all/complete_humanjudge_FV_ChatGPT.json", 'w') as f:
    json.dump(trials_dict,f)


In [ ]:
confmat_overall

{'h_machine': array([[611., 389.],
        [466., 534.]]),
 'h_ivsn': array([[146., 104.]]),
 'h_gbvs': array([[143., 107.]]),
 'h_deepgaze': array([[ 54., 196.]]),
 'h_chatgpt': array([[123., 127.]])}

In [ ]:
np.mean(dummy_marks_overall)/40

np.float64(0.6965)

In [ ]:
(611/(611.+ 389) + 127/(123.+ 127.))/2

0.5595

Bad pipe message: %s [b'\x8f\xe4\x9f\x0fi<\x81\'\xba\x82@b\xb9|\xf3\xf8\x90\x8c\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\x00\x87\x00\x88\x00\x89\x00\x8a\x00\x8b\x00\x8c\x00\x8d\x00\x8e\x00\x8f\x00\x90\x00\x91\x00\x92\x00\x93\x00\x94\x00\x95\x00']
Bad pipe message: %s [b'\x97\x00\x98\x00\x99\x00\x9a\x00\x9b\x00\x9c\x00\x9d\x00\x9e\x00\x9f\x00\xa0\x00\xa1\x00\xa2\x00\xa3\x00\xa4\x00\xa5\x00\xa6\x00\xa7\x00\xa8\x00\xa9\x00\xaa\x00\xab\x00\xac\x00\xad\x00\xae\x00\xaf\x00\xb0\x00\xb1\x00\xb2\x00\xb3\x00\xb4\x00\xb5\x00\xb6\x00\xb7\x00\xb8\

In [ ]:
catch_trial_mat_overall

array([[0., 0.]])

In [ ]:
534/(466.+ 534.)

0.534